In [ ]:
import requests
from shapely.geometry import shape, Point, Polygon, MultiPolygon
from shapely.ops import nearest_points

def get_zip_polygon_osm(zip_code, country_code):
    url = f"https://nominatim.openstreetmap.org/search?postalcode={zip_code}&countrycodes={country_code}&format=geojson&polygon_geojson=1"
    headers = {'User-Agent': 'Mozilla/5.0'}
    res = requests.get(url, headers=headers)

    if res.status_code == 200 and len(res.json()['features']) > 0:
        geom = res.json()['features'][0]['geometry']
        return shape(geom)
    else:
        return None

def calculate_distance_km(geom1, geom2):
    if isinstance(geom1, (Point)) and isinstance(geom2, (Point)):
        return geom1.distance(geom2) * 111  # Aproximación grados a km (latitud)
    elif isinstance(geom1, (Polygon, MultiPolygon)) and isinstance(geom2, (Polygon, MultiPolygon)):
        p1, p2 = nearest_points(geom1.boundary, geom2.boundary)
        return p1.distance(p2) * 111
    elif isinstance(geom1, Point) and isinstance(geom2, (Polygon, MultiPolygon)):
        p1, p2 = nearest_points(geom1, geom2.boundary)
        return p1.distance(p2) * 111
    elif isinstance(geom2, Point) and isinstance(geom1, (Polygon, MultiPolygon)):
        p1, p2 = nearest_points(geom2, geom1.boundary)
        return p1.distance(p2) * 111
    else:
        raise ValueError("Unvalid Geometry")

def are_adjacent(geom1, geom2):
    if isinstance(geom1, (Polygon, MultiPolygon)) and isinstance(geom2, (Polygon, MultiPolygon)):
        return geom1.touches(geom2)
    return False

zip1 = get_zip_polygon_osm("08034", "ES")  # Barcelona
zip2 = get_zip_polygon_osm("08028", "ES")  # Barcelona

if zip1 and zip2:
    dist_km = calculate_distance_km(zip1, zip2)
    if dist_km < 3: 
        print("Adjacent")
    else:
        print("Not adyacent")


Distancia aproximada: 1.97 km
Adjacentes
